# 1. Install prerequisites

In [ ]:
from importlib.metadata import version

In [1]:
version('torch')

'2.4.0'

In [5]:
version('triton')

'3.0.0'

In [ ]:
pip install --upgrade transformers

In [2]:
version('transformers')

'4.44.2'

## vLLM

https://docs.vllm.ai/en/latest/

In [ ]:
pip install --upgrade vllm

In [3]:
version('vllm')

'0.6.1.post2'

## SGLang

https://sglang.readthedocs.io/en/latest/

In [ ]:
pip install --upgrade "sglang[all]"

In [4]:
version('sglang')

'0.3.1.post2'

In [ ]:
pip install flashinfer -i https://flashinfer.ai/whl/cu124/torch2.4/

In [6]:
version('flashinfer')

'0.1.6+cu124torch2.4'

## ollama

https://github.com/ollama/ollama/blob/main/docs/linux.md#manual-install

In [11]:
!mkdir ollama && curl -L https://ollama.com/download/ollama-linux-amd64.tgz -o ollama-linux-amd64.tgz && tar -C ./ollama -xzf ollama-linux-amd64.tgz && rm ollama-linux-amd64.tgz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   117  100   117    0     0    566      0 --:--:-- --:--:-- --:--:--   567
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1583M  100 1583M    0     0  17.1M      0  0:01:32  0:01:32 --:--:-- 16.9M


```bash
./ollama/bin/ollama serve &
```

https://github.com/ollama/ollama-python

https://github.com/ollama/ollama/tree/main/docs

In [ ]:
pip install --upgrade ollama

In [9]:
version('ollama')

'0.3.3'

# 2. Performance testing functions

## Test models

https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct

https://huggingface.co/collections/neuralmagic/llama-31-quantization-66a3f907f48d07feabb8f300

In [1]:
test_models = {                                                                    # OpenLLM leaderboard score
    "llama-3.1" : "meta-llama/Meta-Llama-3.1-8B-Instruct",                         # 100.0 %
    "llama-3.1:w8a16" : "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16",  # 99.8 %    
    "llama-3.1:fp8" : "neuralmagic/Meta-Llama-3.1-8B-Instruct-FP8",                # 99.5 % - warning the "FP8-dynamic" version is MUCH slower on RTX 4090 !
    "llama-3.1:w8a8" : "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a8",    # 99.4 %
    "llama-3.1:w4a16" : "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16"   # 97.1 %
}

Note: to create custom quantized model versions -> https://docs.vllm.ai/en/latest/quantization/fp8.html#quantization-process

https://ollama.com/library/llama3.1

```bash
./ollama/bin/ollama pull llama3.1:8b-instruct-fp16
./ollama/bin/ollama pull llama3.1:8b-instruct-q8_0
./ollama/bin/ollama pull llama3.1:8b-instruct-q4_0
```

In [2]:
ollama_test_models = {
    "llama-3.1" : "llama3.1:8b-instruct-fp16",
    "llama-3.1:int8" : "llama3.1:8b-instruct-q8_0",
    "llama-3.1:int4" : "llama3.1:8b-instruct-q4_0"
}

## Test prompts

In [3]:
test_messages = [
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages du Crédit Mutuel ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages du Crédit Agricole ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la Société Générale ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la BNP ?"}
]
]

In [4]:
from transformers import AutoTokenizer

def format_prompt(messages, model):
    tokenizer = AutoTokenizer.from_pretrained(model)
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

In [5]:
format_prompt(test_messages, test_models["llama-3.1"])

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nTu es un assistant utile et professionnel qui répond toujours en français.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuels sont les avantages du Crédit Mutuel ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
 '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nTu es un assistant utile et professionnel qui répond toujours en français.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuels sont les avantages du Crédit Agricole ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
 '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nTu es un assistant utile et professionnel qui répond toujours en français.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuels sont 

### vLLM

In [6]:
# Authenticate VLLM with Huggingface Hub
import os

with open("/workspace/hftoken", 'r') as file:
    myhftoken = file.read().strip()

os.environ["HF_TOKEN"]=myhftoken

In [7]:
import time
from vllm import LLM, SamplingParams

def vllm_load(model):    
    llm = LLM(model, gpu_memory_utilization=0.99, max_model_len=8192)
    llm._model = model
    return llm

def vllm_generate(messages, llm):    
    print(f"vLLM performance test:")
    
    prompts = format_prompt(messages, llm._model)
    sampling_params = SamplingParams(temperature=0.7, top_p=0.8, repetition_penalty=1.05, max_tokens=512)
    # warmup
    outputs = llm.generate(prompts[0], sampling_params)
    print(f"Generated text: {outputs[0].outputs[0].text!r}")
    
    for batch_size in range(1, len(messages) + 1):
        start_time = time.time()  # Record the start time
        outputs = llm.generate(prompts[0:batch_size], sampling_params)
        end_time = time.time()  # Record the end time
            
        # Print the outputs.
        tokenscount = 0
        for output in outputs:
            generated_text = output.outputs[0].text
            tokenscount = tokenscount + len(output.outputs[0].token_ids)

        tokens_per_sec = tokenscount/(end_time-start_time)
        print(f"- batch size {batch_size}: {tokens_per_sec:.2f} tokens/sec ({batch_size} x {tokens_per_sec/batch_size:.2f})")

In [8]:
llm = vllm_load(test_models["llama-3.1"])

INFO 09-22 14:24:00 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llama/Meta-Llama-3.1-8B-Instruct, use_v2_block_manager=False, num_scheduler_steps=1, en

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 09-22 14:24:17 model_runner.py:1008] Loading model weights took 14.9888 GB
INFO 09-22 14:24:19 gpu_executor.py:122] # GPU blocks: 3610, # CPU blocks: 2048
INFO 09-22 14:24:31 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-22 14:24:31 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-22 14:24:42 model_runner.py:1430] Graph capturing finished in 11 secs.


In [17]:
vllm_generate(test_messages*8, llm)

vLLM performance test:


Processed prompts: 100%|█████████████| 1/1 [00:09<00:00,  9.58s/it, est. speed input: 6.58 toks/s, output: 53.47 toks/s]


Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre plusieurs avantages à ses membres et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés sur les épargnes** : Le Crédit Mutuel propose des taux d'intérêt plus élevés sur les comptes d'épargne que de nombreux autres établissements bancaires.\n2. **Taux d'emprunt compétitifs** : Les prêts personnels, les prêts immobiliers et les prêts pour la mobilité sont proposés avec des taux d'intérêt attractifs.\n3. **Services personnalisés** : Le Crédit Mutuel offre des services personnalisés et adaptés aux besoins de ses membres et clients, grâce à une approche relationnelle et à une connaissance approfondie de leurs situations financières.\n4. **Sécurité et confidentialité** : Le Crédit Mutuel s'engage à protéger les données personnelles et financières de ses membres et clients, conformément aux règles de protection des données.\n5. **Participation aux décisions** : En tant que membre 

Processed prompts: 100%|█████████████| 1/1 [00:09<00:00,  9.30s/it, est. speed input: 6.78 toks/s, output: 55.07 toks/s]


- batch size 1: 55.06 tokens/sec (1 x 55.06)


...

- batch size 30: 1242.63 tokens/sec (30 x 41.42)


Processed prompts: 100%|███████| 31/31 [00:12<00:00,  2.53it/s, est. speed input: 159.80 toks/s, output: 1282.74 toks/s]


- batch size 31: 1282.23 tokens/sec (31 x 41.36)


Processed prompts: 100%|███████| 32/32 [00:12<00:00,  2.56it/s, est. speed input: 161.42 toks/s, output: 1308.09 toks/s]

- batch size 32: 1307.55 tokens/sec (32 x 40.86)


### SGLang

In [6]:
import json, time
import sglang

def sglang_load(model):
    runtime = sglang.Runtime(model_path=model)
    runtime._model = model
    return runtime

def sglang_generate(messages, runtime):
    print(f"SGLang performance test:")
    
    prompts = format_prompt(messages, runtime._model)
    sampling_params = { "temperature":0.7, "top_p":0.8, "repetition_penalty":1.05, "max_new_tokens":512 }
    # warmup
    output = json.loads(runtime.generate(prompt=prompts[0], sampling_params=sampling_params))
    print(f"Generated text: {output['text']!r}")
    
    for batch_size in range(1, len(messages) + 1):
        start_time = time.time()  # Record the start time
        outputs = json.loads(runtime.generate(prompt=prompts[0:batch_size], sampling_params=sampling_params))
        end_time = time.time()  # Record the end time
            
        # Print the outputs.
        tokenscount = 0
        for output in outputs:
            generated_text = output["text"]
            tokenscount = tokenscount + output["meta_info"]["completion_tokens"]

        tokens_per_sec = tokenscount/(end_time-start_time)
        print(f"- batch size {batch_size}: {tokens_per_sec:.2f} tokens/sec ({batch_size} x {tokens_per_sec/batch_size:.2f})")

In [ ]:
runtime = sglang_load(test_models["llama-3.1"])

In [7]:
sglang_generate(test_messages*8, runtime)

SGLang performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre divers avantages à ses membres et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés sur les dépôts** : Le Crédit Mutuel propose des taux d'intérêt plus élevés que les banques traditionnelles pour les dépôts, ce qui permet aux clients de gagner plus d'argent sur leurs économies.\n\n2. **Taux d'emprunt compétitifs** : Les emprunts du Crédit Mutuel sont souvent moins chers que ceux proposés par les banques traditionnelles, ce qui peut aider les clients à se procurer des prêts à des conditions avantageuses.\n\n3. **Services personnalisés** : En tant que banque coopérative, le Crédit Mutuel prend en compte les besoins spécifiques de ses clients et offre des services personnalisés pour répondre à leurs attentes.\n\n4. **Transparence et sécurité** : Le Crédit Mutuel est connu pour sa transparence financière et sa sécurité, ce qui rassure les clients qui s

### Ollama

https://github.com/ollama/ollama-python

In [45]:
import ollama

ollama.list()

{'models': [{'name': 'llama3.1:8b-instruct-q4_0',
   'model': 'llama3.1:8b-instruct-q4_0',
   'modified_at': '2024-09-22T11:43:03.484852591+02:00',
   'size': 4661230766,
   'digest': '42182419e9508c30c4b1fe55015f06b65f4ca4b9e28a744be55008d21998a093',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '8.0B',
    'quantization_level': 'Q4_0'}},
  {'name': 'llama3.1:8b-instruct-q8_0',
   'model': 'llama3.1:8b-instruct-q8_0',
   'modified_at': '2024-09-22T11:33:10.006555339+02:00',
   'size': 8540789934,
   'digest': 'b158ded76fa05be6bce8a682099ce5df8c5571340a04cf63a2923464679db576',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '8.0B',
    'quantization_level': 'Q8_0'}},
  {'name': 'llama3.1:8b-instruct-fp16',
   'model': 'llama3.1:8b-instruct-fp16',
   'modified_at': '2024-09-21T22:54:30.926572546+02:00',
   'size': 16068910253

In [27]:
print(ollama.show("llama3.1:8b-instruct-fp16")['template'])

{{- if or .System .Tools }}<|start_header_id|>system<|end_header_id|>
{{- if .System }}

{{ .System }}
{{- end }}
{{- if .Tools }}

Cutting Knowledge Date: December 2023

When you receive a tool call response, use the output to format an answer to the orginal user question.

You are a helpful assistant with tool calling capabilities.
{{- end }}<|eot_id|>
{{- end }}
{{- range $i, $_ := .Messages }}
{{- $last := eq (len (slice $.Messages $i)) 1 }}
{{- if eq .Role "user" }}<|start_header_id|>user<|end_header_id|>
{{- if and $.Tools $last }}

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.

{{ range $.Tools }}
{{- . }}
{{ end }}
Question: {{ .Content }}<|eot_id|>
{{- else }}

{{ .Content }}<|eot_id|>
{{- end }}{{ if $last }}<|start_header_id|>assistant<|end_header_id|>

{{ e

https://github.com/ollama/ollama/blob/main/docs/api.md#generate-a-completion

In [41]:
import ollama

# ollama keeps models 5 min in memory by default, they are reloaded by a query
def ollama_load(model):
    sampling_params = { "num_predict":1 }
    ollama.generate(model=model, prompt="load", raw=True, options=sampling_params, stream=False)
    return model

# ollama API only supports batch size 1
def ollama_generate(messages, model):
    print(f"ollama performance test:")
    
    prompts = format_prompt(messages, runtime._model)
    sampling_params = { "temperature":0.7, "top_p":0.8, "repeat_penalty":1.05, "num_predict":512 }
    # warmup
    output = ollama.generate(model=model, prompt=prompts[0], raw=True, options=sampling_params, stream=False)
    print(f"Generated text: {output['response']!r}")
    
    for msg_index in range(len(messages)):
        start_time = time.time()  # Record the start time
        output = ollama.generate(model=model, prompt=prompts[msg_index], raw=True, options=sampling_params, stream=False)
        end_time = time.time()  # Record the end time
            
        # Print the outputs.
        tokenscount = output['eval_count']
        tokens_per_sec = tokenscount/(end_time-start_time)
        print(f"- batch size 1: {tokens_per_sec:.2f} tokens/sec")

In [43]:
model = ollama_load(ollama_test_models["llama-3.1"])

In [44]:
ollama_generate(test_messages, model)

ollama performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre divers avantages à ses adhérents. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts réduits sur les prêts** : Le Crédit Mutuel propose des taux d'intérêt compétitifs pour les prêts personnels, hypothécaires et professionnels.\n2. **Avantages fiscaux** : Les adhérents du Crédit Mutuel peuvent bénéficier de réductions fiscales sur leurs prêts et épargnes, en fonction de leur situation financière et de leurs impôts.\n3. **Assistance financière** : Le Crédit Mutuel offre des services d'assistance financière pour aider ses adhérents à gérer leurs finances et à atteindre leurs objectifs économiques.\n4. **Produits diversifiés** : Le Crédit Mutuel propose une gamme complète de produits bancaires, tels que les comptes courants, les livrets d'épargne, les cartes de crédit, les assurances et les investissements.\n5. **Services en ligne** : Les adhérents du Crédit Mutuel ont accè

# 3. Performance tests on RTX 4090

## vLLM

### FP16

In [7]:
llm = vllm_load(test_models["llama-3.1"])

INFO 09-22 12:18:56 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llama/Meta-Llama-3.1-8B-Instruct, use_v2_block_manager=False, num_scheduler_steps=1, en

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 09-22 12:19:07 model_runner.py:1008] Loading model weights took 14.9888 GB
INFO 09-22 12:19:08 gpu_executor.py:122] # GPU blocks: 3610, # CPU blocks: 2048
INFO 09-22 12:19:08 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-22 12:19:08 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-22 12:19:18 model_runner.py:1430] Graph capturing finished in 10 secs.


In [8]:
vllm_generate(test_messages*12, llm)

vLLM performance test:


Processed prompts: 100%|█████████████| 1/1 [00:09<00:00,  9.62s/it, est. speed input: 6.55 toks/s, output: 53.23 toks/s]


Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre plusieurs avantages à ses membres et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés sur les épargnes** : Le Crédit Mutuel propose des taux d'intérêt plus élevés sur les comptes d'épargne que de nombreux autres établissements bancaires.\n2. **Taux d'emprunt compétitifs** : Les prêts personnels, les prêts immobiliers et les prêts pour la mobilité sont proposés avec des taux d'intérêt attractifs.\n3. **Services personnalisés** : Le Crédit Mutuel offre des services personnalisés et adaptés aux besoins de ses membres et clients, grâce à une approche relationnelle et à une connaissance approfondie de leurs situations financières.\n4. **Sécurité et confidentialité** : Le Crédit Mutuel s'engage à protéger les données personnelles et financières de ses membres et clients, conformément aux règles de protection des données.\n5. **Participation aux décisions** : En tant que membre 

Processed prompts: 100%|█████████████| 1/1 [00:09<00:00,  9.29s/it, est. speed input: 6.78 toks/s, output: 55.11 toks/s]


- batch size 1: 55.10 tokens/sec (1 x 55.10)


...

- batch size 46: 1070.79 tokens/sec (46 x 23.28)


Processed prompts: 100%|███████| 47/47 [00:22<00:00,  2.13it/s, est. speed input: 134.17 toks/s, output: 1087.55 toks/s]


- batch size 47: 1087.21 tokens/sec (47 x 23.13)


Processed prompts: 100%|███████| 48/48 [00:22<00:00,  2.13it/s, est. speed input: 134.46 toks/s, output: 1085.58 toks/s]

- batch size 48: 1085.12 tokens/sec (48 x 22.61)


### w8a16

In [8]:
llm = vllm_load(test_models["llama-3.1:w8a16"])

INFO 09-22 12:47:04 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16', speculative_config=None, tokenizer='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=compressed-tensors, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=neuralmagic/Meta-Llama-3.1-8B-Instruct-quant

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-22 12:47:08 model_runner.py:1008] Loading model weights took 8.4927 GB
INFO 09-22 12:47:09 gpu_executor.py:122] # GPU blocks: 6716, # CPU blocks: 2048
INFO 09-22 12:47:09 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-22 12:47:09 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-22 12:47:18 model_runner.py:1430] Graph capturing finished in 9 secs.


In [10]:
vllm_generate(test_messages*18, llm)

vLLM performance test:


Processed prompts: 100%|█████████████| 1/1 [00:06<00:00,  6.05s/it, est. speed input: 6.94 toks/s, output: 84.63 toks/s]


Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre de nombreux avantages à ses adhérents et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés** : Le Crédit Mutuel propose des taux d'intérêt plus élevés que les banques traditionnelles, notamment pour les comptes courants et les prêts.\n2. **Services personnalisés** : En tant que banque coopérative, le Crédit Mutuel met l'accent sur la proximité et la personnalisation de ses services. Les clients ont accès à des conseillers financiers compétents qui leur proposent des solutions adaptées à leurs besoins.\n3. **Transparence et sécurité** : Le Crédit Mutuel est connu pour sa transparence dans les tarifs et les conditions de prêt. Les clients sont également protégés par des garanties de sécurité renforcées.\n4. **Épargne et placement** : Le Crédit Mutuel propose une gamme de produits d'épargne et de placement attractifs, notamment des comptes d'épargne, des livrets et des fonds 

Processed prompts: 100%|█████████████| 1/1 [00:05<00:00,  5.82s/it, est. speed input: 7.21 toks/s, output: 87.92 toks/s]


- batch size 1: 87.89 tokens/sec (1 x 87.89)


...

- batch size 67: 2457.61 tokens/sec (67 x 36.68)


Processed prompts: 100%|███████| 68/68 [00:14<00:00,  4.75it/s, est. speed input: 199.54 toks/s, output: 2413.53 toks/s]


- batch size 68: 2412.29 tokens/sec (68 x 35.47)


Processed prompts:   0%|                     | 0/69 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

KeyboardInterrupt: 

### fp8

Note: the "FP8-dynamic" version is much slower that the "FP8" version.
- batch size 16: 61 tokens/sec vs 77 tokens/sec
- batch size 32: 42 tokens/sec vs 62 tokens/sec
- batch size 48: 03 tokens/sec vs 52 tokens/sec !

In [8]:
llm = vllm_load(test_models["llama-3.1:fp8"])

config.json:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

INFO 09-22 13:39:33 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='neuralmagic/Meta-Llama-3.1-8B-Instruct-FP8', speculative_config=None, tokenizer='neuralmagic/Meta-Llama-3.1-8B-Instruct-FP8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=compressed-tensors, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=neuralmagic/Meta-Llama-3.1-8B-Instruct-FP8, use_v2_block_manager=Fal

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.8k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-22 13:47:57 model_runner.py:1008] Loading model weights took 8.4889 GB
INFO 09-22 13:47:58 gpu_executor.py:122] # GPU blocks: 6856, # CPU blocks: 2048
INFO 09-22 13:47:58 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-22 13:47:58 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-22 13:48:07 model_runner.py:1430] Graph capturing finished in 9 secs.


In [9]:
vllm_generate(test_messages*16, llm)

vLLM performance test:


Processed prompts: 100%|████████████| 1/1 [00:06<00:00,  6.11s/it, est. speed input: 10.32 toks/s, output: 83.84 toks/s]


Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre plusieurs avantages à ses membres et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés sur les épargnes** : Le Crédit Mutuel propose des taux d'intérêt plus élevés sur les comptes d'épargne comparés aux banques traditionnelles.\n2. **Frais de gestion réduits** : Les frais de gestion pour les comptes courants et les prêts sont souvent moins élevés chez le Crédit Mutuel que dans les banques conventionnelles.\n3. **Prêts personnalisés** : Le Crédit Mutuel prend en compte la situation personnelle et financière de chaque client pour proposer des prêts adaptés à ses besoins.\n4. **Soutien à l'entrepreneuriat** : Le Crédit Mutuel propose des solutions financières spécifiques pour les entrepreneurs et les PME, notamment des prêts et des services de financement.\n5. **Services bancaires complets** : Le Crédit Mutuel propose une gamme complète de services bancaires, y compris les co

Processed prompts: 100%|████████████| 1/1 [00:05<00:00,  5.93s/it, est. speed input: 10.62 toks/s, output: 86.31 toks/s]


- batch size 1: 86.27 tokens/sec (1 x 86.27)


...

- batch size 65: 972.07 tokens/sec (65 x 14.95)


Processed prompts: 100%|████████| 66/66 [00:34<00:00,  1.92it/s, est. speed input: 120.88 toks/s, output: 978.95 toks/s]


- batch size 66: 978.71 tokens/sec (66 x 14.83)


Processed prompts:   0%|                     | 0/67 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

KeyboardInterrupt: 

### w8a8

In [8]:
llm = vllm_load(test_models["llama-3.1:w8a8"])

config.json:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

INFO 09-22 14:08:30 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a8', speculative_config=None, tokenizer='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=compressed-tensors, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=neuralmagic/Meta-Llama-3.1-8B-Instruct-quantiz

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/43.5k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-22 14:16:47 model_runner.py:1008] Loading model weights took 8.4939 GB
INFO 09-22 14:16:48 gpu_executor.py:122] # GPU blocks: 6759, # CPU blocks: 2048
INFO 09-22 14:16:48 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-22 14:16:48 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-22 14:16:57 model_runner.py:1430] Graph capturing finished in 9 secs.


In [9]:
vllm_generate(test_messages*18, llm)

vLLM performance test:


Processed prompts: 100%|█████████████| 1/1 [00:06<00:00,  6.90s/it, est. speed input: 6.09 toks/s, output: 74.24 toks/s]


Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre plusieurs avantages à ses membres et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus faibles** : Le Crédit Mutuel propose des taux d'intérêt compétitifs pour les prêts, les comptes courants et les épargnes.\n2. **Accès à des services bancaires complets** : Le Crédit Mutuel offre une gamme complète de services bancaires, y compris des prêts, des comptes courants, des épargnes, des cartes de crédit, des assurances et des investissements.\n3. **Conseils personnalisés** : Les conseillers du Crédit Mutuel peuvent vous aider à gérer votre patrimoine, à élaborer un plan financier et à atteindre vos objectifs économiques.\n4. **Transparence et sécurité** : Le Crédit Mutuel est une banque coopérative, ce qui signifie que les décisions sont prises collectivement par les membres et non par des actionnaires. Cela assure une transparence et une sécurité supplémentaires.\n5. **Services en li

Processed prompts: 100%|█████████████| 1/1 [00:06<00:00,  6.63s/it, est. speed input: 6.33 toks/s, output: 77.21 toks/s]


- batch size 1: 77.18 tokens/sec (1 x 77.18)


...

- batch size 31: 337.58 tokens/sec (31 x 10.89)


Processed prompts: 100%|█████████| 32/32 [00:59<00:00,  1.87s/it, est. speed input: 22.40 toks/s, output: 269.94 toks/s]


- batch size 32: 269.92 tokens/sec (32 x 8.44)


Processed prompts:   0%|                     | 0/33 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

KeyboardInterrupt: 

### w4a16

In [8]:
llm = vllm_load(test_models["llama-3.1:w4a16"])

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

INFO 09-22 14:25:08 gptq_marlin.py:108] The model is convertible to gptq_marlin during runtime. Using gptq_marlin kernel.
INFO 09-22 14:25:08 llm_engine.py:223] Initializing an LLM engine (v0.6.1.post2) with config: model='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16', speculative_config=None, tokenizer='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_tim

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

WARNING 09-22 14:25:10 utils.py:727] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
INFO 09-22 14:25:10 model_runner.py:997] Starting to load model neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w4a16...
INFO 09-22 14:25:11 weight_utils.py:242] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/5.74G [00:00<?, ?B/s]

INFO 09-22 14:30:25 weight_utils.py:287] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 09-22 14:30:26 model_runner.py:1008] Loading model weights took 5.3812 GB
INFO 09-22 14:30:28 gpu_executor.py:122] # GPU blocks: 8325, # CPU blocks: 2048
INFO 09-22 14:30:30 model_runner.py:1311] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-22 14:30:30 model_runner.py:1315] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-22 14:30:40 model_runner.py:1430] Graph capturing finished in 10 secs.


In [9]:
vllm_generate(test_messages*18, llm)

vLLM performance test:


Processed prompts: 100%|████████████| 1/1 [00:04<00:00,  4.38s/it, est. speed input: 9.58 toks/s, output: 116.78 toks/s]


Generated text: "Le Crédit Mutuel est une banque mutuelle française qui offre divers avantages à ses clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts sur les comptes courants** : Le Crédit Mutuel offre des intérêts sur les comptes courants, ce qui permet aux clients de gagner de l'argent sans avoir à investir leurs fonds.\n2. **Prêt à usage** : Le Crédit Mutuel propose des prêts à usage avec des taux d'intérêt compétitifs, ce qui peut aider les clients à financer leurs besoins ou leurs projets.\n3. **Assurance de prêt** : Le Crédit Mutuel offre une assurance de prêt qui couvre les risques de défaut de remboursement, ce qui donne aux emprunteurs une sécurité supplémentaire.\n4. **Services bancaires complets** : Le Crédit Mutuel propose une gamme complète de services bancaires, y compris des cartes de crédit, des chèques, des virements et des paiements en ligne.\n5. **Sécurité et protection** : Le Crédit Mutuel met en place des mesures de sécurité pour protéger les

Processed prompts: 100%|███████████| 1/1 [00:04<00:00,  4.08s/it, est. speed input: 10.30 toks/s, output: 125.62 toks/s]


- batch size 1: 125.55 tokens/sec (1 x 125.55)


...

- batch size 70: 2874.84 tokens/sec (70 x 41.07)


Processed prompts: 100%|███████| 71/71 [00:11<00:00,  5.99it/s, est. speed input: 251.64 toks/s, output: 2927.52 toks/s]


- batch size 71: 2925.52 tokens/sec (71 x 41.20)


Processed prompts: 100%|███████| 72/72 [00:12<00:00,  5.91it/s, est. speed input: 248.35 toks/s, output: 2909.87 toks/s]

- batch size 72: 2907.97 tokens/sec (72 x 40.39)


## SGLang

### fp16

In [7]:
runtime = sglang_load(test_models["llama-3.1"])

INFO 09-22 14:43:27 weight_utils.py:242] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


In [8]:
sglang_generate(test_messages*8, runtime)

SGLang performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre divers avantages à ses membres, notamment :\n\n1. **Intérêts plus élevés sur les épargnes** : Les membres du Crédit Mutuel bénéficient souvent de taux d'intérêt plus élevés que ceux proposés par les banques traditionnelles sur leurs comptes d'épargne.\n2. **Prêts à des conditions favorables** : Les membres peuvent obtenir des prêts à des taux d'intérêt attractifs et avec des conditions de remboursement flexibles.\n3. **Sécurité et stabilité** : En tant que banque coopérative, le Crédit Mutuel est contrôlée par ses membres, ce qui garantit une gestion responsable et durable de l'argent.\n4. **Services personnalisés** : Les conseillers du Crédit Mutuel prennent le temps de comprendre les besoins de chaque membre pour offrir des solutions adaptées à leur situation financière.\n5. **Économies sur les frais** : Les membres du Crédit Mutuel peuvent bénéficier de réductions sur les frais 

### w8a16

In [7]:
runtime = sglang_load(test_models["llama-3.1:w8a16"])

INFO 09-22 14:52:36 weight_utils.py:242] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Process Process-1:2:
Traceback (most recent call last):
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/workspace/wordslab-llms/.venv/lib/python3.11/site-packages/sglang/srt/managers/detokenizer_manager.py", line 185, in start_detokenizer_process
    loop.run_until_complete(manager.handle_loop())
  File "uvloop/loop.pyx", line 1511, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1504, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1377, in uvloop.loop.Loop.run_forever
  File "uvloop/loop.pyx", line 555, in uvloop.loop.Loop._run
Process Process-1:1:
Traceback (most recent call last):
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 314, in _bootstra

In [8]:
sglang_generate(test_messages*10, runtime)

SGLang performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre plusieurs avantages à ses adhérents et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts attractifs** : Le Crédit Mutuel propose des taux d'intérêt compétitifs pour les prêts, les comptes courants et les épargnes.\n2. **Services personnalisés** : En tant que banque coopérative, le Crédit Mutuel se concentre sur la relation avec ses adhérents et clients, offrant des services personnalisés et adaptés à leurs besoins.\n3. **Sécurité et confidentialité** : Comme banque coopérative, le Crédit Mutuel est soumis à des règles de confidentialité et de sécurité strictes pour protéger les données personnelles et financières de ses adhérents et clients.\n4. **Prêts sans frais de dossier** : Le Crédit Mutuel propose des prêts sans frais de dossier pour certains types de prêts, ce qui peut aider les clients à économiser de l'argent.\n5. **Cartes de crédit et de débit** : Le

KeyboardInterrupt: 

### fp8

In [ ]:
runtime = sglang_load(test_models["llama-3.1:fp8"])

**INTERNAL ERROR** during FP8 model load

### w8a8

In [7]:
runtime = sglang_load(test_models["llama-3.1:w8a8"])

INFO 09-22 15:04:44 weight_utils.py:242] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Process Process-1:2:
Traceback (most recent call last):
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/workspace/wordslab-llms/.venv/lib/python3.11/site-packages/sglang/srt/managers/detokenizer_manager.py", line 185, in start_detokenizer_process
    loop.run_until_complete(manager.handle_loop())
  File "uvloop/loop.pyx", line 1511, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1504, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1377, in uvloop.loop.Loop.run_forever
  File "uvloop/loop.pyx", line 555, in uvloop.loop.Loop._run
  File "uvloop/handles/poll.pyx", line 216, in uvloop.loop.__on_uvpoll_event
  File "uvloop/cbhandles.pyx", line 83, in uvloop.loop.Handle._run
  File "uvloop/cbhandles.py

In [8]:
sglang_generate(test_messages*8, runtime)

SGLang performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre divers avantages à ses adhérents et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts attractifs** : Le Crédit Mutuel propose des taux d'intérêt compétitifs pour les prêts, les comptes courants et les épargnes.\n2. **Service personnalisé** : En tant que banque coopérative, le Crédit Mutuel se concentre sur la relation avec ses clients et adhérents, offrant un service personnalisé et attentif.\n3. **Produits financiers adaptés** : Le Crédit Mutuel propose une large gamme de produits financiers adaptés aux besoins de ses clients, tels que les prêts immobiliers, les prêts personnels, les comptes courants, les comptes d'épargne, etc.\n4. **Sécurité et protection** : Le Crédit Mutuel met en place des mesures de sécurité et de protection pour protéger les données et les comptes de ses clients.\n5. **Transparence et responsabilité** : En tant que banque coopérative, 

KeyboardInterrupt: 

### w4a16

In [7]:
runtime = sglang_load(test_models["llama-3.1:w4a16"])

INFO 09-22 15:13:32 gptq_marlin.py:108] The model is convertible to gptq_marlin during runtime. Using gptq_marlin kernel.
INFO 09-22 15:13:33 weight_utils.py:242] Using model weights format ['*.safetensors']
INFO 09-22 15:13:33 weight_utils.py:287] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Process Process-1:2:
Traceback (most recent call last):
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/workspace/wordslab-llms/.venv/lib/python3.11/site-packages/sglang/srt/managers/detokenizer_manager.py", line 185, in start_detokenizer_process
    loop.run_until_complete(manager.handle_loop())
  File "uvloop/loop.pyx", line 1511, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1504, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1377, in uvloop.loop.Loop.run_forever
  File "uvloop/loop.pyx", line 555, in uvloop.loop.Loop._run
  File "uvloop/handles/poll.pyx", line 216, in uvloop.loop.__on_uvpoll_event
  File "uvloop/cbhandles.pyx", line 83, in uvloop.loop.Handle._run
  File "uvloop/cbhandles.py

In [8]:
sglang_generate(test_messages*18, runtime)

SGLang performance test:
Generated text: "Le Crédit Mutuel est une banque mutuelle française qui offre divers services financiers à ses clients. Voici quelques-uns des principaux avantages du Crédit Mutuel :\n\n1. **Sécurité et stabilité** : Le Crédit Mutuel est une banque mutuelle, ce qui signifie qu'il est géré par ses membres (clients) et non par des actionnaires. Cela garantit sa stabilité et sa sécurité.\n2. **Services personnalisés** : Le Crédit Mutuel propose des services personnalisés adaptés aux besoins de ses clients, qu'ils soient particuliers ou professionnels.\n3. **Offres de produits variées** : La banque propose une gamme de produits et services, notamment des comptes courants, des épargnes, des crédits, des assurances et des services d'investissement.\n4. **Prêt rapide et à bon taux d'intérêt** : Le Crédit Mutuel offre des prêts à des taux d'intérêt compétitifs et avec des conditions de remboursement flexibles.\n5. **Assistance et conseil** : Les clients du Crédit Mutue

KeyboardInterrupt: 